# Battery Simulator

This notebook simulates home battery systems to optimize energy costs based on day-ahead pricing data.


## 1. Imports and Setup


In [1]:
import pandas as pd
from datetime import datetime
from typing import Dict
import altair as alt
import pulp

# Import from our battery simulator module
from battery_simulator import BatterySimulator, BatteryConfig, plot_hourly_battery_operation, plot_profit_by_period

# Configure Altair
alt.data_transformers.disable_max_rows()
alt.renderers.enable("mimetype")


RendererRegistry.enable('mimetype')

## 2. Battery System Constants

Configure the parameters for your battery system here. The parameters are currently set to a 628Ah, 16-cell, 48V battery architecture and a Victron MultiPlus-II charger/inverter.

In [2]:
BATTERY_CAPACITY_WH = 628 * 3.45 * 16
CHARGE_RATE_W = 4000
DISCHARGE_RATE_W = 3000

CHARGE_EFFICIENCY = 0.90
DISCHARGE_EFFICIENCY = 0.90

INITIAL_SOC = 0.10
MIN_SOC = 0.10
MAX_SOC = 0.90

# Economics (for wear optimization)
BATTERY_COST_EUR = 1500  # Total battery replacement cost
BATTERY_CYCLE_LIFE = 6000  # Rated cycle life to 80% capacity
COST_PER_CYCLE = BATTERY_COST_EUR / BATTERY_CYCLE_LIFE

BATTERY_CONFIG: BatteryConfig = {
    "capacity_wh": BATTERY_CAPACITY_WH,
    "charge_rate_w": CHARGE_RATE_W,
    "discharge_rate_w": DISCHARGE_RATE_W,
    "charge_efficiency": CHARGE_EFFICIENCY,
    "discharge_efficiency": DISCHARGE_EFFICIENCY,
    "min_soc": MIN_SOC,
    "max_soc": MAX_SOC,
    "initial_soc": INITIAL_SOC,
    "cost_per_cycle": COST_PER_CYCLE,
}

## 3. Algorithms

The below cell defines algorithms that can be pushed into the BatterySimulator to check their real-world performance.


In [3]:
def lp_algorithm(
    current_time: datetime,
    battery_soc: float,
    known_prices: pd.DataFrame,
    battery_specs: BatteryConfig,
    state: Dict = None
):
    """
    Optimal battery scheduling using Linear Programming.
    
    Strategy:
    - At 13:00 daily: solve an LP to find globally optimal charge/discharge schedule
    - Objective: Maximize profit (revenue from discharge - cost of charge)
    - Constraints:
      * Battery SOC continuity (energy balance over time)
      * SOC limits (min_soc to max_soc)
      * Power limits (max charge/discharge rates)
      * Efficiency losses (charge_efficiency, discharge_efficiency)
      * Cannot charge and discharge simultaneously
    
    This finds the mathematically optimal solution within the planning horizon,
    unlike heuristics which use rules of thumb.
    
    Returns:
        Tuple of (desired_power_w, new_state)
    """
    
    if state is None:
        state = {}
    
    current_hour = current_time.hour
    
    # Solve optimization at 13:00 or if we don't have a schedule
    if current_hour == 13 or 'schedule' not in state or state.get('last_schedule_date') != current_time.date():
        # Get all known prices (up to 48 hours at 13:00)
        future_prices = known_prices[known_prices['datetime'] >= current_time].copy()
        
        if len(future_prices) < 2:
            return 0.0, state
        
        # Extract battery parameters
        capacity_wh = battery_specs['capacity_wh']
        charge_rate_w = battery_specs['charge_rate_w']
        discharge_rate_w = battery_specs['discharge_rate_w']
        charge_eff = battery_specs['charge_efficiency']
        discharge_eff = battery_specs['discharge_efficiency']
        min_soc = battery_specs['min_soc']
        max_soc = battery_specs['max_soc']
        
        # Time horizon
        T = len(future_prices)
        times = list(range(T))
        
        # Create LP problem - maximize profit (minimize cost)
        prob = pulp.LpProblem("Battery_Optimization", pulp.LpMinimize)
        
        # Decision variables
        # charge[t] = energy charged from grid in hour t (kWh, >= 0)
        charge = pulp.LpVariable.dicts("charge", times, lowBound=0, upBound=charge_rate_w/1000)
        
        # discharge[t] = energy discharged to grid in hour t (kWh, >= 0)
        discharge = pulp.LpVariable.dicts("discharge", times, lowBound=0, upBound=discharge_rate_w/1000)
        
        # soc[t] = state of charge at END of hour t (fraction, 0-1)
        soc = pulp.LpVariable.dicts("soc", times, lowBound=min_soc, upBound=max_soc)
        
        # Binary variables to prevent simultaneous charge/discharge
        is_charging = pulp.LpVariable.dicts("is_charging", times, cat='Binary')
        is_discharging = pulp.LpVariable.dicts("is_discharging", times, cat='Binary')
        
        # Variables for cycle cost calculation (linearization of |SOC[t] - SOC[t-1]|)
        soc_delta = pulp.LpVariable.dicts("soc_delta", times, lowBound=0)
        
        # Objective: minimize cost (negative revenue = maximize profit) + cycle wear cost
        # Cost = sum over time of (charge_cost - discharge_revenue + cycle_cost)
        prices = future_prices['price'].values
        cost_per_cycle = battery_specs.get('cost_per_cycle', 0.0)
        usable_range = max_soc - min_soc
        
        # Convert cycle cost from "per full cycle" to "per SOC unit"
        cycle_cost_per_soc_unit = cost_per_cycle / usable_range if usable_range > 0 else 0
        
        prob += pulp.lpSum([
            prices[t] * charge[t] - prices[t] * discharge[t] + cycle_cost_per_soc_unit * soc_delta[t]
            for t in times
        ]), "Total_Cost"
        
        # Constraints
        for t in times:
            # SOC continuity constraint
            if t == 0:
                # Initial SOC is current battery state
                energy_in = charge[t] * charge_eff * 1000  # Convert kWh to Wh
                energy_out = discharge[t] / discharge_eff * 1000
                prob += soc[t] == battery_soc + (energy_in - energy_out) / capacity_wh
                
                # Cycle cost: delta from initial state
                prob += soc_delta[t] >= soc[t] - battery_soc
                prob += soc_delta[t] >= battery_soc - soc[t]
            else:
                # SOC evolves based on previous SOC + charge - discharge
                energy_in = charge[t] * charge_eff * 1000
                energy_out = discharge[t] / discharge_eff * 1000
                prob += soc[t] == soc[t-1] + (energy_in - energy_out) / capacity_wh
                
                # Cycle cost: |SOC[t] - SOC[t-1]| linearization
                prob += soc_delta[t] >= soc[t] - soc[t-1]
                prob += soc_delta[t] >= soc[t-1] - soc[t]
            
            # Cannot charge and discharge at the same time
            prob += is_charging[t] + is_discharging[t] <= 1
            
            # Link binary variables to continuous variables
            # If not charging, charge must be 0
            prob += charge[t] <= is_charging[t] * (charge_rate_w / 1000)
            
            # If not discharging, discharge must be 0
            prob += discharge[t] <= is_discharging[t] * (discharge_rate_w / 1000)
        
        # Solve the LP
        prob.solve(pulp.PULP_CBC_CMD(msg=0))  # Silent solver
        
        # Extract optimal schedule
        schedule = {}
        for t in times:
            dt = future_prices.iloc[t]['datetime']
            charge_val = pulp.value(charge[t]) or 0
            discharge_val = pulp.value(discharge[t]) or 0
            
            if charge_val > 0.001:  # Small threshold for numerical stability
                schedule[dt] = ('charge', charge_val * 1000)  # Convert back to Wh
            elif discharge_val > 0.001:
                schedule[dt] = ('discharge', discharge_val * 1000)
            else:
                schedule[dt] = ('idle', 0)
        
        state['schedule'] = schedule
        state['last_schedule_date'] = current_time.date()
        state['solver_status'] = pulp.LpStatus[prob.status]
    
    # Execute scheduled action for current hour
    action, energy_wh = state['schedule'].get(current_time, ('idle', 0))
    
    if action == 'charge' and battery_soc < battery_specs['max_soc'] - 0.01:
        # Return power in watts (rate for 1 hour = energy in Wh)
        return float(min(energy_wh, battery_specs['charge_rate_w'])), state
    
    elif action == 'discharge' and battery_soc > battery_specs['min_soc'] + 0.01:
        # Negative for discharge
        return -float(min(energy_wh, battery_specs['discharge_rate_w'])), state
    
    else:
        return 0.0, state


In [4]:
# Run Linear Programming algorithm
simulator_lp = BatterySimulator(BATTERY_CONFIG)
results_lp = simulator_lp.run(lp_algorithm)


Simulation complete: 50874 hours processed


In [5]:
# Visualize LP algorithm operation
lp_chart = plot_hourly_battery_operation(results_lp, "2024-12-10", days=4)
lp_chart


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [6]:
print(f"Total profit: €{-results_lp['grid_cost_eur'].sum():.2f}")
print(f"Total Battery Cycles: {results_lp['cycle_fraction'].sum():.1f} cycles")

Total profit: €2556.00
Total Battery Cycles: 2192.8 cycles


In [7]:
# Weekly profit comparison for LP algorithm
chart_lp_profit = plot_profit_by_period(
    results_lp,
    period="month",
    start_date="2020-01-01",
    end_date="2025-10-19",
)

chart_lp_profit

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting
